In [1]:
import numpy as np
import pandas as pd
import ee
import folium
from folium import plugins
import json
import geopandas as gpd
import matplotlib as mpl
import fiona
import datetime
from csv import writer

In [2]:
# ee.Authenticate()
ee.Initialize()

In [42]:
# computation

def manualNDWI(img, b3 = 'B3', b8 = 'B8'):
    return img.select(b3).subtract(img.select(b8)).divide(img.select(b3).add(img.select(b3)))

def manualFAPAR(img, b4 = 'B4', b8 = 'B8'):
    return img.select(b8).subtract(img.select(b4)).divide(img.select(b8).add(img.select(b4)))

def manualNDVI(img, b5 = 'B5', b4 = 'B4'):
    return img.select(b5).subtract(img.select(b4)).divide(img.select(b5).add(img.select(b4)))
        
def manualStag(NDWI_img, FAPAR_img, threshold=0):
  return NDWI_img.multiply(FAPAR_img).gt(threshold)

def manualNDMI(img, b5 = 'B5', b6 = 'B6'):
  return img.select(b5).subtract(img.select(b6)).divide(img.select(b5).add(img.select(b6)))

def manualAerosol(img, aerosol = 'SR_QA_AEROSOL'):
  return img. select(aerosol).rename('aerosol')

def manualSurfTemp(img, SurfTemp = 'LST_Day_1km'):
  return img.select(SurfTemp).rename('surface_temparature')

In [69]:
#key: layername, value1: layer, value2: wrap
cities = {
    # 'CAVITE': '14.3294,120.9367',
    'ZAMBOANGA_CITY': '6.9167,122.0833',
    'LAGUNA': '14.2167,121.1667',
    'CEBU': '10.3,123.9',
    'ILOILO': '10.7167,122.5667',
    'QUEZON_CITY': '14.6333,121.0333',
    'PANGASINAN': '15.9281,120.3489',
    'BULACAN': '14.7928,120.8789',
    'NEGROS_OCCIDENTAL': '9.9833,122.8167',
    'BATANGAS': '13.75,121.05',
    'RIZAL': '14.5842,121.1763',
    'PAMPANGA': '15.0333,120.6833',
    'BUKIDNON': '7.9,125.0833',
    'NUEVA_ECIJA': '15.4833,120.9667',
    'TARLAC': '15.4802,120.5979',
    'NEGROS ORIENTAL': '9.3103,123.3081',
    'BOHOL': '9.65,123.85',
    'SOUTH_COTABATO': '6.5,124.85',
    'QUEZON': '7.7333,125.1',
}

def latlong_gen(city, diff):
    latlong = cities[city]
    lat_v, long_v = latlong.split(",")
    
    #define size of BBOX?
    LL_lat = round(float(lat_v) - diff,4)
    LL_long = round(float(long_v) - diff,4)
    UR_lat = round(float(lat_v) + diff, 4)
    UR_long = round(float(long_v) + diff, 4)

    bbox = [LL_lat, LL_long, UR_lat, UR_long] 
    
    return bbox

In [71]:
SatCode = 'COPERNICUS/S2_SR'
sat = ee.ImageCollection(SatCode)
diff = 0.11
start_date = "2019-01-01"
end_date = "2022-08-30"

start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end = datetime.datetime.strptime(end_date, '%Y-%m-%d')
days = abs(start-end).days
total_weeks = days//7

for city, shapefile in cities.items():
    bbox = latlong_gen(city, diff)
    gee_bbox = ee.Geometry.Rectangle(bbox)
    geometry = ee.Geometry(ee.FeatureCollection(gee_bbox).geometry())
    print(city)

    for week in range(total_weeks):
        error_code = ''

        current_date = (start.date() + datetime.timedelta(days = 7 * week)).isoformat()
        current_end_date = (start.date() + datetime.timedelta(days = 7 * week) + datetime.timedelta(days = 7)).isoformat()
        print(current_date, current_end_date)
        img = sat.filterBounds(geometry).filterDate(current_date, current_end_date)

        try:
            # get NDWI
            # compute index
            print('getting NDWI for date', current_date, current_end_date)
            ndwi = manualNDWI(img.median())
            # get mean values
            avg_ndwi = ndwi.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=geometry,
                        scale=50,
                        bestEffort=True
                    ).getInfo()
            avg_ndwi = list(avg_ndwi.values())[0]
            
        except Exception as e: 
            print(e)
            avg_ndwi = None
            error_code = error_code + str(e) + " | "

        try:
            # get FAPAR
            # compute index
            print('getting FAPAR for date', current_date, current_end_date)
            fapar = manualFAPAR(img.median())

            avg_fapar = fapar.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=geometry,
                        scale=50,
                        bestEffort=True
                    ).getInfo()

            avg_fapar = list(avg_fapar.values())[0]

        except Exception as e: 
            print(e)
            avg_fapar = None
            error_code = error_code + str(e) + " | "
        try:
            # get NDVI
            # compute index
            print('getting NDVI for date', current_date, current_end_date)
            ndvi=manualNDVI(img.median())
            avg_ndvi = ndvi.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=geometry,
                        scale=50,
                        bestEffort=True
                    ).getInfo()

            ndvi=manualNDVI(img.median())
            avg_ndvi = list(avg_ndvi.values())[0]

        except Exception as e: 
            print(e)
            avg_ndvi = None
            error_code = error_code + str(e) + " | "

        res = [city, avg_ndwi, avg_fapar, avg_ndvi, current_date, current_end_date, error_code]
                    
        with open('ee_location_newtest.csv', 'a') as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(res)
            f_object.close()
        print(res)

ZAMBOANGA_CITY
2019-01-01 2019-01-08
getting NDWI for date 2019-01-01 2019-01-08
getting FAPAR for date 2019-01-01 2019-01-08
getting NDVI for date 2019-01-01 2019-01-08
['ZAMBOANGA_CITY', 0.028074487185745983, -0.07588092342025092, 0.010038852407184488, '2019-01-01', '2019-01-08', '']
2019-01-08 2019-01-15
getting NDWI for date 2019-01-08 2019-01-15
getting FAPAR for date 2019-01-08 2019-01-15
getting NDVI for date 2019-01-08 2019-01-15
['ZAMBOANGA_CITY', -0.030576630307925786, 0.017953682828643503, 0.006069162953949269, '2019-01-08', '2019-01-15', '']
2019-01-15 2019-01-22
getting NDWI for date 2019-01-15 2019-01-22
getting FAPAR for date 2019-01-15 2019-01-22
getting NDVI for date 2019-01-15 2019-01-22
['ZAMBOANGA_CITY', 0.01602692108428095, -0.03221023073762776, -0.0015688642386176138, '2019-01-15', '2019-01-22', '']
2019-01-22 2019-01-29
getting NDWI for date 2019-01-22 2019-01-29
getting FAPAR for date 2019-01-22 2019-01-29


bbox = latlong_gen('CEBU', diff)
bbox

bbox = latlong_gen('CEBU', diff)
gee_aoi = ee.Geometry.Rectangle(bbox)
print(gee_aoi)

start_date = "2020-10-01"
end_date = "2020-10-08"

start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end = datetime.datetime.strptime(end_date, '%Y-%m-%d')

bbox = latlong_gen('ZAMBOANGA_CITY', diff)
gee_aoi = ee.Geometry.Rectangle(bbox)
SatCode = 'COPERNICUS/S2_SR'
geometry = ee.Geometry(ee.FeatureCollection(gee_aoi).geometry())
sat = ee.ImageCollection(SatCode)
img = sat.filterBounds(geometry).filterDate(start, end)

def manualNDWI(img, b3 = 'B3', b8 = 'B8'):
    return img.select(b3).subtract(img.select(b8)).divide(img.select(b3).add(img.select(b3)))

ndwi = manualNDWI(img.median())

avg_ndwi = ndwi.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=geometry,
                        scale=30,
                        bestEffort=True
                    ).getInfo()

avg_ndwi = list(avg_ndwi.values())[0]
avg_ndwi

avg_ndwi

error_code = None 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0cd7ca08-c036-4633-8a95-c1f919155399' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>